<a href="https://colab.research.google.com/github/sathishsara1/IntelliCheck/blob/main/TextPlagly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL NECESSARY LIBRARIES

In [36]:
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2 nltk transformers torch scikit-learn


UPLOAD PDF

In [40]:
from google.colab import files

uploaded = files.upload()


Saving hihello.pdf to hihello.pdf


EXTRACT TEXT FROM PDF

In [41]:
import PyPDF2

file_name = list(uploaded.keys())[0]
pdf_reader = PyPDF2.PdfReader(file_name)

extracted_text = ""
for page in pdf_reader.pages:
    extracted_text += page.extract_text()

print(extracted_text[:500])


INTELLICHECK – A PLAGIARISM DETECTOR FOR SOURCE CODE AND TEXT 
USING NATURAL LANGUAGE PROCESSING, HIDDEN MARKOV MODEL AND 
TRANSFORMER MODELS 
ABSTRACT : 
In today’s digital age, ensuring the originality of academic work, code, and content is of 
paramount importance, necessitating the development of robust plagiarism detection 
systems. This paper explores the use of advanced machine learning techniques such as 
Natural Language Processing (NLP), Hidden Markov Models (HMM), and Transformer-
bas


PREPROCESS THE TEXT

In [42]:
import re

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text) #Extra Characters
    text = re.sub(r'\s+', ' ', text) #Extra Spaces
    text = text.lower() #converting into lower case
    text = text.strip() #removing leading and trailing spaces
    return text


In [43]:
processed_text = preprocess_text(extracted_text)
print(processed_text[:500])


intellicheck a plagiarism detector for source code and text using natural language processing hidden markov model and transformer models abstract in todays digital age ensuring the originality of academic work code and content is of paramount importance necessitating the development of robust plagiarism detection systems this paper explores the use of advanced machine learning techniques such as natural language processing nlp hidden markov models hmm and transformer based models like bert to de


LOADING THE CORPUS

In [ ]:
import os

extracted_path = "/content/Corpus"

file_names = []
for root, dirs, files in os.walk(extracted_path):
    for file in files:
        file_names.append(os.path.join(root, file))

print("Files in the folder:")
for name in file_names:
    print(name)


In [ ]:
import os

source_dir = "/content/Corpus"
source_files = []
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith(".txt"):
            source_files.append(os.path.join(root, file))
print(f"Total source documents found: {len(source_files)}")


In [22]:
source_texts = []
idx = 0
for file in source_files[:100]:
    with open(file, 'r', encoding='utf-8') as f:
        source_text = f.read()
        source_texts.append(preprocess_text(source_text))

print(f"Processed first 100 source documents.")


Processed first 100 source documents.


HMM MODEL DEVELOPMENT (HIDDEN MARKOV MODEL)

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(3, 3))
source_matrix = vectorizer.fit_transform(source_texts)

query_matrix = vectorizer.transform([processed_text])

from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(query_matrix, source_matrix)

print(f"Top 5 structural similarities:")
top_matches = similarity_scores[0].argsort()[-5:][::-1]
for idx in top_matches:
    print(f"Match with {source_files[idx]} - Similarity: {similarity_scores[0][idx] * 100:.2f}%")


Top 5 structural similarities:
Match with /content/Corpus/Corpus/source-document01327.txt - Similarity: 5.59%
Match with /content/Corpus/Corpus/source-document01653.txt - Similarity: 5.24%
Match with /content/Corpus/Corpus/source-document00540.txt - Similarity: 4.78%
Match with /content/Corpus/Corpus/source-document01382.txt - Similarity: 4.77%
Match with /content/Corpus/Corpus/source-document01698.txt - Similarity: 4.41%


BERT MODEL

In [45]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

query_embedding = model.encode([processed_text])
source_embeddings = model.encode(source_texts[:10])

import numpy as np
semantic_similarities = np.inner(query_embedding, source_embeddings)

print(f"Top 5 semantic similarities:")
top_semantic_matches = np.argsort(semantic_similarities[0])[-5:][::-1]
for idx in top_semantic_matches:
    print(f"Match with {source_files[idx]} - Similarity: {semantic_similarities[0][idx] :.2f}%")


Top 5 semantic similarities:
Match with /content/Corpus/Corpus/source-document01244.txt - Similarity: 3.84%
Match with /content/Corpus/Corpus/source-document00808.txt - Similarity: 3.79%
Match with /content/Corpus/Corpus/source-document01327.txt - Similarity: 3.28%
Match with /content/Corpus/Corpus/source-document01113.txt - Similarity: 2.92%
Match with /content/Corpus/Corpus/source-document00861.txt - Similarity: 2.38%


In [27]:
print("Size of semantic_similarities:", len(semantic_similarities[0]))
print("Indices in top_matches:", top_matches)


Size of semantic_similarities: 10
Indices in top_matches: [ 7 92 74 63 70]


OVERALL PLAGIARISM SCORE

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer(ngram_range=(3, 3))
source_matrix = vectorizer.fit_transform(source_texts)

query_matrix = vectorizer.transform([processed_text])

similarity_scores = cosine_similarity(query_matrix, source_matrix)

N = 20
top_indices = similarity_scores[0].argsort()[-N:][::-1]
top_scores = similarity_scores[0][top_indices]

structural_plagiarism_score = (top_scores.sum())
print(f"Plagiarism Score (Structural Similarity): {structural_plagiarism_score:.2f}%")


Plagiarism Score (Structural Similarity): 0.60%


In [59]:
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

query_embedding = model.encode([processed_text])
source_embeddings = model.encode(source_texts)
semantic_similarities = np.inner(query_embedding, source_embeddings)

N = 20
top_indices = np.argsort(semantic_similarities[0])[-N:][::-1]
top_scores = semantic_similarities[0][top_indices]
print(top_scores)
semantic_plagiarism_score = (top_scores.sum())
print(f"Plagiarism Score (Semantic Similarity): {semantic_plagiarism_score:.2f}%")


[5.7607327 5.4016914 5.111039  4.8841214 4.659977  4.290891  4.2390966
 4.1816425 4.161416  3.9998739 3.8390465 3.7971458 3.790763  3.6983268
 3.647286  3.6363814 3.602428  3.5520313 3.4950533 3.4918807]
Plagiarism Score (Semantic Similarity): 83.24%


In [61]:
weight_hmm = 0.2
weight_bert = 0.8


overall_plagiarism = ((weight_hmm * structural_plagiarism_score) + (weight_bert * semantic_plagiarism_score))

print("\nOverall Scores:")
print(f"Overall Plagiarism: {overall_plagiarism:.2f}%")



Overall Scores:
Overall Plagiarism: 66.71%
